In [1]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
#from torchvision import datasets, transforms

In [2]:
#learning_rate = 1e-3
learning_rate = 0.2

batch_size = 1
epochs = 5

In [3]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(f'Using {device} device')

Using cpu device


In [4]:
class MyData(Dataset):
    def __init__(self, file_name, device):
        'Initialization'
        d = pd.read_csv(file_name)
        x = d.iloc[0:,0:5].values
        y = d.iloc[0:,5].values

        self.x = torch.tensor(x, dtype=torch.float32).to(device)
        self.y = torch.tensor(y, dtype=torch.float32).to(device)
        
    def __len__(self):
        'Denotes the total number of samples'
        #return len(self.list_IDs)
        return len(self.y)

    def __getitem__(self, index):
        'Generates one sample of data'
        return self.x[index], self.y[index]

In [5]:
training_data = MyData('train.csv', device)
test_data = MyData('test.csv', device)

train_dataloader = DataLoader(training_data, batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size, shuffle=True)

In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 1),
#            nn.ReLU(),
            #nn.Linear(10, 1),
        )

    def forward(self, x):
        x = self.linear_relu_stack(x)
        return x

In [12]:
#model = NeuralNetwork()
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=1, bias=True)
  )
)


In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



In [9]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

In [13]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.000691  [    0/10000]
loss: 0.000000  [ 1000/10000]
loss: 0.000000  [ 2000/10000]
loss: 0.000000  [ 3000/10000]
loss: 0.000000  [ 4000/10000]
loss: 0.000000  [ 5000/10000]
loss: 0.000000  [ 6000/10000]
loss: 0.000000  [ 7000/10000]
loss: 0.000000  [ 8000/10000]
loss: 0.000000  [ 9000/10000]
Test Error: 
 Avg loss: 0.000000 

Epoch 2
-------------------------------
loss: 0.000000  [    0/10000]
loss: 0.000000  [ 1000/10000]
loss: 0.000000  [ 2000/10000]
loss: 0.000000  [ 3000/10000]
loss: 0.000000  [ 4000/10000]
loss: 0.000000  [ 5000/10000]
loss: 0.000000  [ 6000/10000]
loss: 0.000000  [ 7000/10000]
loss: 0.000000  [ 8000/10000]
loss: 0.000000  [ 9000/10000]
Test Error: 
 Avg loss: 0.000000 

Epoch 3
-------------------------------
loss: 0.000000  [    0/10000]
loss: 0.000000  [ 1000/10000]
loss: 0.000000  [ 2000/10000]
loss: 0.000000  [ 3000/10000]
loss: 0.000000  [ 4000/10000]
loss: 0.000000  [ 5000/10000]
loss: 0.000000  [ 6000/10000]


In [16]:
input_data = torch.rand(1,10)
print(input_data.size())

torch.Size([1, 10])


In [18]:
X = torch.rand(1, 10, device=device)
logits = model(X)
#pred_probab = nn.Softmax(dim=1)(logits)
#y_pred = pred_probab.argmax(1)
print(f"Predicted class: {logits}")

Predicted class: tensor([[-0.0120]], device='cuda:0', grad_fn=<AddmmBackward0>)
